## VideoStream Visualization

In [2]:
from rikai.types import VideoStream

video = VideoStream("elephants_dream.mp4")
video

VideoStream(uri=elephants_dream.mp4)

## SparkSession Initialization

In [1]:
from rikai.spark.utils import init_spark_session
from rikai.spark.functions import init

spark = init_spark_session(
    dict(
        [
            (
                "spark.rikai.sql.ml.registry.torchhub.impl",
                "ai.eto.rikai.sql.model.torchhub.TorchHubRegistry",
            )
        ]
    )
)

init(spark)

:: loading settings :: url = jar:file:/Users/da/.pyenv/versions/3.8.10/envs/rikai-example/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-71e0311a-f90f-4034-b954-4c0650d36070;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.0.21 in central
	found org.antlr#antlr4-runtime;4.8-1 in local-m2-cache
	found com.thoughtworks.enableIf#enableif_2.12;1.1.7 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found io.circe#circe-core_2.12;0.12.3 in central
	found io.circe#circe-numbers_2.12;0.12.3 in central
	found org.typelevel#cats-core_2.12;2.0.0 in central
	found org.typelevel#cats-macros_2.12;2.0.0 in central
	found org.typelevel#cats-kernel_2.12;2.0.0 in central
	found io.circe#circe-generic_2.12;0.12.3 in central
	found com.chuusai#shapeless_2.12;2.3.3 in spark-list
	found org.typeleve

In [2]:
spark.sql("""
CREATE MODEL yolov5m
FLAVOR yolov5 
OPTIONS (device="cpu", batch_size=32)
USING "torchhub:///ultralytics/yolov5:v6.0/yolov5m";
""")

DataFrame[]

## Case 1: Apply ML_PREDICT on Elephants Dream

In [3]:
result = spark.sql("""
from (
    select 
        frame_id,
        date_format(ts, 'mm:ss.SSS') as ts,
        to_image(image_data) as image
    from video.`elephants_dream.mp4`
    where frame_id > 1000 and frame_id < 1100
)
select
    frame_id,
    ts,
    image,
    ML_PREDICT(yolov5m, image) as preds
limit 10
""").collect()

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'file:///Users/da/da-tubi/rikai-example/elephants_dream.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: mp41
    creation_time   : 1970-01-01T00:00:00.000000Z
    title           : Elephants Dream - http://www.archive.org/details/ElephantsDream
    artist          : Bassam Kurdali
    composer        : Bassam Kurdali
    date            : 2006
    encoder         : Lavf51.10.0
    comment         : license:  http://creativecommons.org/licenses/by/3.0/us/
  Duration: 00:10:53.79, start: 0.000000, bitrate: 575 kb/s
  Stream #0:0(und): Video: h264 (Constrained Baseline) (avc1 / 0x31637661), yuv420p, 426x240 [SAR 1:1 DAR 71:40], 507 kb/s, 24 fps, 24 tbr, 24 tbn, 48 tbc (default)
    Metadata:
      creation_time   : 1970-01-01T00:00:00.000000Z
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 64 kb/s (

In [4]:
result[0]

Row(frame_id=1001, ts='00:41.708', image=Image(<embedded>), preds=[Row(box=Box2d(xmin=326.5046691894531, ymin=56.99554443359375, xmax=376.2666931152344, ymax=187.65403747558594), score=0.8062904477119446, label='person'), Row(box=Box2d(xmin=77.05941009521484, ymin=197.52145385742188, xmax=153.72970581054688, ymax=239.360107421875), score=0.33259767293930054, label='chair'), Row(box=Box2d(xmin=65.09420776367188, ymin=144.2378692626953, xmax=159.7301025390625, ymax=179.4154510498047), score=0.25948402285575867, label='person')])

In [5]:
result[0].image

Image(<embedded>)

In [7]:
from rikai.types.geometry import Box2d
box = result[0].preds[0].box

result[0].image | box

Draw(Box2d(xmin=326.5046691894531, ymin=56.99554443359375, xmax=376.2666931152344, ymax=187.65403747558594))

## Case 2: 画面中包含人类的帧占所有帧的比例

+ 假设1：视频画面中包含人类的帧的占比，和该视频受欢迎程度相关
+ 影视剧采购优化：优先采购人类帧占比在[x, y]区间之内的视频

In [10]:
spark.sql("""
create or replace temporary view frames
using video
options (
  path "elephants_dream.mp4",
  fps 1
)
""")

DataFrame[]

In [20]:
spark.sql("""
from (
    from (
        from frames
        select 
            frame_id,
            date_format(ts, 'mm:ss.SSS') as ts,
            to_image(image_data) as image,
            ML_PREDICT(yolov5m, to_image(image_data)) as preds
    )
    select
        array_contains(transform(preds, pred -> pred.label), 'person') as is_person
)
select
    is_person,
    count(*) as count
group by is_person
""").toPandas()

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'file:///Users/da/da-tubi/rikai-example/elephants_dream.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: mp41
    creation_time   : 1970-01-01T00:00:00.000000Z
    title           : Elephants Dream - http://www.archive.org/details/ElephantsDream
    artist          : Bassam Kurdali
    composer        : Bassam Kurdali
    date            : 2006
    encoder         : Lavf51.10.0
    comment         : license:  http://creativecommons.org/licenses/by/3.0/us/
  Duration: 00:10:53.79, start: 0.000000, bitrate: 575 kb/s
  Stream #0:0(und): Video: h264 (Constrained Baseline) (avc1 / 0x31637661), yuv420p, 426x240 [SAR 1:1 DAR 71:40], 507 kb/s, 24 fps, 24 tbr, 24 tbn, 48 tbc (default)
    Metadata:
      creation_time   : 1970-01-01T00:00:00.000000Z
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 64 kb/s (

,is_person,count
0,True,337
1,False,317


In [23]:
spark.sql("""
from (
    from (
        from frames
        select 
            frame_id,
            date_format(ts, 'mm:ss.SSS') as ts,
            to_image(image_data) as image,
            ML_PREDICT(yolov5m, to_image(image_data)) as preds
        where
            frame_id > 2000 and frame_id < 3000
    )
    select
        array_contains(transform(preds, pred -> pred.label), 'person') as is_person,
        *
)
select
    is_person,
    frame_id,
    ts,
    image
where
    is_person is false
limit 3
""").toPandas()

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'file:///Users/da/da-tubi/rikai-example/elephants_dream.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: mp41
    creation_time   : 1970-01-01T00:00:00.000000Z
    title           : Elephants Dream - http://www.archive.org/details/ElephantsDream
    artist          : Bassam Kurdali
    composer        : Bassam Kurdali
    date            : 2006
    encoder         : Lavf51.10.0
    comment         : license:  http://creativecommons.org/licenses/by/3.0/us/
  Duration: 00:10:53.79, start: 0.000000, bitrate: 575 kb/s
  Stream #0:0(und): Video: h264 (Constrained Baseline) (avc1 / 0x31637661), yuv420p, 426x240 [SAR 1:1 DAR 71:40], 507 kb/s, 24 fps, 24 tbr, 24 tbn, 48 tbc (default)
    Metadata:
      creation_time   : 1970-01-01T00:00:00.000000Z
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 64 kb/s (

,is_person,frame_id,ts,image
0,False,2004,01:23.500,Image(<embedded>)
1,False,2028,01:24.500,Image(<embedded>)
2,False,2052,01:25.500,Image(<embedded>)


## Quiz 1
+ 调整yolov5模型的参数，得到更加正确的结果

https://github.com/Tubitv/rikai-yolov5

## Quiz 2
+ 计算出现的每一帧中，人类所占部分的面积
+ 假设：视频中人类所占部分的面积占比，和该视频收欢迎程度相关